In [ ]:
import os
import PIL
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
import pathlib

import folium
import base64
from folium import IFrame
from io import BytesIO

In [ ]:
def decimal_coords(coords, ref):
    decimal_degrees = coords[0] + coords[1] / 60.0 + coords[2] / 3600.0
    if ref == "S" or ref == "W":
        decimal_degrees = -decimal_degrees
    return decimal_degrees

In [ ]:
def get_exif(file_name):
    image = PIL.Image.open(file_name)
    return image.getexif()


def get_geo(exif):
    for key, value in TAGS.items():
        if value == "GPSInfo":
            break
    if value != "GPSInfo":
        raise Exception()
    gps_info = exif.get_ifd(key)
    return {GPSTAGS.get(key, key): value for key, value in gps_info.items()}


def get_exif_old(image_file_path):
    exif_table = {}
    image = PIL.Image.open(image_file_path)
    info = image.getexif()

    for tag, value in info.items():
        decoded = TAGS.get(tag, tag)
        exif_table[decoded] = value

    return exif_table

In [ ]:
base_dir = pathlib.Path(r"/home/krzysztof/tmp/wtst")

In [ ]:
def get_coords(img_path):
    try:
        geo = get_geo(get_exif(img_path))
        return [
            decimal_coords(geo["GPSLatitude"], geo["GPSLatitudeRef"]),
            decimal_coords(geo["GPSLongitude"], geo["GPSLongitudeRef"]),
        ]
    except Exception:
        return None

In [ ]:
for img in os.listdir(base_dir):
    # if pathlib.Path(img).suffix == ".pdf":
    #     print("pdf")
    #     continue

    if pathlib.Path(img).suffix.lower() not in [".jpg", ".jpeg"]:
        print("not jpg")
        continue
    
    p = base_dir / img

    print(get_exif_old(p).get("Model"), img)
    try:
        print(get_geo(get_exif(p)))
    except Exception as e:
        print(get_exif_old(p))

In [ ]:
def add_marker(p, m):
    coords = get_coords(p)

    if not coords:
        return m

    buffered = BytesIO()

    image = (
        Image.open(p)
        .resize((400, 350), Image.LANCZOS)
        .save(buffered, format="JPEG")
    )

    encoded = base64.b64encode(buffered.getvalue())

    html = '<img src="data:image/png;base64,{}">'.format
    iframe = IFrame(html(encoded.decode("UTF-8")), width=400, height=350)
    popup = folium.Popup(iframe, max_width=400)

    folium.Marker(
        location=coords,
        popup=popup,
    ).add_to(m)

    return m

In [ ]:
m = folium.Map(location=[40.7128, -74.0060], zoom_start=12)

for p in [os.path.join(base_dir, p) for p in os.listdir(base_dir)]:
    if pathlib.Path(p).suffix.lower() not in [".jpg", ".jpeg"]:
        continue
    if not p :
        continue
    m = add_marker(p, m)

m

In [ ]:
m.save("pops.html")